In [1]:
import torch
from imagen_pytorch import Unet, Imagen, ImagenTrainer
from imagen_pytorch.data import Dataset
import random
from tqdm import tqdm
import numpy as np
from imagen_pytorch.t5 import t5_encode_text

np.random.seed(0)
random.seed(0)
torch.manual_seed(0)
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')

NOTE: Redirects are currently not supported in Windows or MacOs.
C:\Python310\lib\site-packages\beartype\_util\hint\pep\utilpeptest.py:345: BeartypeDecorHintPep585DeprecationWarning: PEP 484 type hint typing.List[str] deprecated by PEP 585 scheduled for removal in the first Python version released after October 5th, 2025. To resolve this, import this hint from "beartype.typing" rather than "typing". See this discussion for further details and alternatives:
    https://github.com/beartype/beartype#pep-585-deprecations
  warn(


In [6]:
## NOTE: Make sure all instances of expm1(x) are purged from the source code (and replaced with a simple exp(x) - 1)
# expm1 is not supported by onnx

In [2]:
unet = Unet(
    dim = 128, # the "Z" layer dimension, i.e. the number of filters the outputs to the first layer
    cond_dim = 256,
    dim_mults = (1, 2, 4), # the channel dimensions inside the model (multiplied by dim)
    num_resnet_blocks = 3,
    layer_attns = (False, True, True),
    layer_cross_attns = (False, True, True)
)

imagen = Imagen(
    unets = unet,
    image_sizes = 32,
    timesteps = 1000,
    cond_drop_prob = 0.1,
    dynamic_thresholding=False
)


class ImagenOnnx(torch.nn.Module):
    def __init__(self, unet, imagen):
        super().__init__()
        self.unet = unet
        self.imagen = imagen

    def sample(self, text_embeds, text_mask, cond_scale,  device, use_tqdm=True):
        batch_size = text_embeds.shape[0]
        noise_scheduler = self.imagen.noise_schedulers[0]
        img = torch.randn((batch_size, self.imagen.sample_channels[0], self.imagen.image_sizes[0], self.imagen.image_sizes[0]), device = device)
        timesteps = noise_scheduler.get_sampling_timesteps(batch_size, device = device)

        for times, times_next in tqdm(timesteps, desc='sampling loop time step', total=len(timesteps), disable=not use_tqdm):       

            img, _ = self.imagen.p_sample(
                self.unet,
                img,
                times,
                t_next = times_next,
                text_embeds = text_embeds,
                text_mask = text_mask,
                cond_scale = cond_scale,
                noise_scheduler = noise_scheduler,
                pred_objective = self.imagen.pred_objectives[0],
                dynamic_threshold = False
            )

        img.clamp_(-1., 1.)

        unnormalize_img = self.imagen.unnormalize_img(img)

        return img
    
    def forward(self, img, text_embeds, text_mask, times, times_next, cond_scale):
        return self.imagen.p_sample(
            self.unet,
            img,
            times,
            t_next = times_next,
            text_embeds = text_embeds,
            text_mask = text_mask,
            cond_scale = cond_scale,
            noise_scheduler=imagen.noise_schedulers[0],
            pred_objective = imagen.pred_objectives[0],
            dynamic_threshold = False
        )
    
u = U(unet, imagen)

In [3]:
# u.sample(torch.rand(1, 10, 768), torch.ones(1, 10, dtype=bool), 1., device)

In [4]:
# u.forward(torch.rand(1, 3, 32, 32),  torch.rand(1, 27, 786), torch.rand(1, 27, 786), torch.Tensor([0.5]), torch.Tensor([0.56]), torch.Tensor([1.]))

In [5]:
torch.onnx.export(
    u, 
    (torch.rand(1, 3, 32, 32),  torch.rand(1, 27, 768), torch.ones(1, 27, dtype=bool), torch.Tensor([0.5]), torch.Tensor([0.56]), torch.Tensor([1.])), 
    "unet.onnx", 
    input_names=['image', 'text_embeds', 'text_mask', 'timestep', 'time_next', 'cond_scale'], 
    output_names=['prediction'],
    dynamic_axes={
        'image': {0: 'batch_size'},
        'text_embeds': {0: 'batch_size', 1: 'n_tokens'},
        'text_mask': {0: 'batch_size', 1: 'n_tokens'},
    }
)

C:\Python310\lib\site-packages\imagen_pytorch\imagen_pytorch.py:2006: TracerWarning: Converting a tensor to a Python boolean might cause the trace to be incorrect. We can't record the data flow of Python values, so this value will be treated as a constant in the future. This means that the trace might not generalize to other inputs!
  assert not (cond_scale != 1. and not self.can_classifier_guidance), 'imagen was not trained with conditional dropout, and thus one cannot use classifier free guidance (cond_scale anything other than 1)'
C:\Python310\lib\site-packages\einops\einops.py:204: UserWarning: __floordiv__ is deprecated, and its behavior will change in a future version of pytorch. It currently rounds toward 0 (like the 'trunc' function NOT 'floor'). This results in incorrect rounding for negative values. To keep the current behavior, use torch.div(a, b, rounding_mode='trunc'), or for actual floor division, use torch.div(a, b, rounding_mode='floor').
  inferred_length: int = length